In [286]:
import requests
import pandas as pd
import pickle

mainpage = 'https://www.immobiliare.it'

In [287]:
def save_obj(obj, name):
    """
    save the object int a pickle file
    """
    with open('data/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    """
    load the object from a pickle file
    """
    with open('data/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [ ]:
from time import time

In [278]:
from bs4 import BeautifulSoup

counter=0

ann_list = []

start = time()
for i in range(1,10001):
    
    # request the html page
    content = requests.get("https://www.immobiliare.it/vendita-case/roma/?criterio=rilevanza&pag=%i"%i)
    # soupify it
    soup = BeautifulSoup(content.text, "lxml")

    
    # find all the tags li with class 'listing-item vetrina js-row-detail'
    announcements = soup.find_all('li', class_ = 'listing-item vetrina js-row-detail')
    if len(announcements) == 0:
        announcements = soup.find_all('li', class_ = 'listing-item js-row-detail')

    
    for ann in announcements:
        title = ann.find('p', class_ = 'titolo text-primary').a['title']
        link = ann.find('p', class_ = 'titolo text-primary').a['href']
        
        locali, superficie, bagni, piani = 'NA','NA','NA','NA'

        if not link.startswith('https://www.immobiliare.it'):
            link = mainpage + link
        subcontent = requests.get(link)
        subsoup = BeautifulSoup(subcontent.text, "html.parser")
        infos = subsoup.find('div', class_='im-property__features')

    
        
        price = infos.find('span', class_='features__price--double')
        if price is None:
            price = infos.find('ul', class_='features__price-block').find('li', class_='features__price').span
        price = price.getText()

        subinfos = infos.find('ul', class_='list-inline list-piped features__list')


        for list_item in subinfos.find_all('li'):
            feature = list_item.find('div', class_='features__label')

            if feature is not None:
                if feature.getText() == 'locali':
                    locali = list_item.find('div').find('span').getText()
                elif feature.getText() == 'superficie':
                    superficie = list_item.find('div').find('span').getText()
                elif feature.getText() == 'bagni':
                    bagni = list_item.find('div').find('span').getText()
                elif feature.getText() == 'piani':
                    piani = list_item.find('div').find('span').getText()

                
        description = subsoup.find('div', class_='left-side')
        description = description.find('div', class_='col-xs-12 description-text text-compressed').div.getText()

        attr_list=[title, price, locali, superficie, bagni, piani, description]
        ann_list.append(attr_list)
        counter=counter+1
        if counter==10:
            break
        

end = time()

In [272]:

cols=['title', 'price', 'locali', 'superficie', 'bagni', 'piani', 'description']
df = pd.DataFrame(ann_list, columns=cols)

In [285]:
prova = load_obj('prova')
prova

,title,price,locali,superficie,bagni,piani,description
0,Nuovi Appartamenti e Attici / Mansarde in Vend...,€ 192.000 - € 705.000,1 - 5,46,NA,NA,\n Le residenze...
1,"Bilocale viale Italo Calvino, Fonte Ostiense, ...",€ 225.000,2,50,1,NA,\n PAPILLO EUR ...
2,"Quadrilocale via Mario Ridolfi 32, Pisana - Br...",€ 359.000,4,130,3,NA,\n Rif: 18100 -...
3,"Appartamento via dei Gandolfi, Camilluccia - F...",€ 800.000,5+,220,3,NA,\n In strada pr...
4,"Villa via Casalmonferrato, Re di Roma, Roma",€ 3.400.000,5+,500,3+,NA,\n Nel quartier...
5,"Appartamento piazza Cavour, Prati, Roma",€ 950.000,5,170,1,NA,"\n Prati, appar..."
6,"Quadrilocale viale della Grande Muraglia 540, ...",€ 250.000,4,110,2,NA,"\n Eur, apparta..."
7,"Appartamento piazza Del Fante, Mazzini - Delle...",€ 1.090.000,5+,190,2,NA,\n In un contes...
8,"Attico / Mansarda via della Frezza, Piazza del...",€ 2.150.000,4,150,1,NA,\n Nel contesto...
9,"Appartamento via Dandolo, Trastevere, Roma",€ 2.500.000,5+,300,3,NA,\n Nella splend...


In [215]:


locali, superficie, bagni, piani = 'NA','NA','NA','NA'


if not link.startswith('https://www.immobiliare.it'):
    link = mainpage + link
subcontent = requests.get(link)
subsoup = BeautifulSoup(subcontent.text, "html.parser")
infos = subsoup.find('div', class_='im-property__features')

price = infos.find('span', class_='features__price--double')
if price is None:
    price = infos.find('ul', class_='features__price-block').find('li', class_='features__price').span
price = price.getText()

subinfos = infos.find('ul', class_='list-inline list-piped features__list')


for list_item in subinfos.find_all('li'):
    feature = list_item.find('div', class_='features__label')
    
    if feature is not None:
        if feature.getText() == 'locali':
            locali = list_item.find('div').find('span').getText()
        elif feature.getText() == 'superficie':
            superficie = list_item.find('div').find('span').getText()
        elif feature.getText() == 'bagni':
            bagni = list_item.find('div').find('span').getText()
        elif feature.getText() == 'piani':
            piani = list_item.find('div').find('span').getText()

description = subsoup.find('div', class_='left-side')
description = description.find('div', class_='col-xs-12 description-text text-compressed').div.getText()


attr_list=[price, locali, superficie, bagni, piani]

ann_list.append(attr_list)



In [222]:
#infos.find('ul', class_='list-inline list-piped features__list')

In [241]:

ann_list = []

locali, superficie, bagni, piani = 'NA','NA','NA','NA'


if not link.startswith('https://www.immobiliare.it'):
    link = mainpage + link
subcontent = requests.get(link)
subsoup = BeautifulSoup(subcontent.text, "html.parser")
infos = subsoup.find('div', class_='im-property__features')

price = infos.find('span', class_='features__price--double')
if price is None:
    price = infos.find('ul', class_='features__price-block').find('li', class_='features__price').span
price = price.getText()

subinfos = infos.find('ul', class_='list-inline list-piped features__list')


for list_item in subinfos.find_all('li'):
    feature = list_item.find('div', class_='features__label')
    
    if feature is not None:
        if feature.getText() == 'locali':
            locali = list_item.find('div').find('span').getText()
        elif feature.getText() == 'superficie':
            superficie = list_item.find('div').find('span').getText()
        elif feature.getText() == 'bagni':
            bagni = list_item.find('div').find('span').getText()
        elif feature.getText() == 'piani':
            piani = list_item.find('div').find('span').getText()

description = subsoup.find('div', class_='left-side')
description = description.find('div', class_='col-xs-12 description-text text-compressed').div.getText()


attr_list=[price, locali, superficie, bagni, piani]

ann_list.append(attr_list)

In [242]:
attr_lst = []

'\n                                box al piano secondo seminterrato edificio A del complesso Levante di 44 mq in ottimo stato vendesi\n            '

In [221]:
print(locali)
print(superficie)
print(bagni)
print(piani)

NA
44
NA
NA
